In [ ]:
%load_ext autoreload
%autoreload 2

import os
import cv2
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()
from research.utils.data_access_utils import RDSAccessUtils
from research_lib.utils.data_access_utils import S3AccessUtils
from tracker_utils import process_annotated_crops, overlay_crops_on_image, stitch_frames_into_video, OUTPUT_DIR
from PIL import Image


<h1> Load annotated data </h1>

In [ ]:
plali_sql_credentials = {
    'host': 'aquabyte-research-db.cfwlu7jbdcqj.eu-west-1.rds.amazonaws.com',
    'port': 5432,
    'user': 'plali',
    'password': 'amethyst that lift dune',
    'database': 'aquabyte_prod'
}

rds = RDSAccessUtils(plali_sql_credentials)
s3 = S3AccessUtils('/root/data')

In [ ]:
query = """
    select * from plali.plali_annotations x
    inner join 
    ( select a.id as plali_image_id, a.images, a.priority, a.metadata, b.id as workflow_id, b.name from plali.plali_images a
    inner join plali.plali_workflows b
    on a.workflow_id = b.id ) y
    on x.plali_image_id = y.plali_image_id
    where workflow_id = '00000000-0000-0000-0000-000000000057';
"""

df = rds.extract_from_database(query).sort_values('priority', ascending=False)

<h1> Generate video with head annotations overlayed </h1>

In [ ]:
output_fs = []

count = 0
for idx, row in tqdm(df.iterrows()):
    anns = [ann for ann in row.annotation['annotations'] if ann['category'] == 'HEAD']
    processed_anns = process_annotated_crops(anns)
    image_s3_url = row.images[0]
    image_s3_url_components = image_s3_url.replace('s3://', '').split('/')
    bucket, key = image_s3_url_components[0], os.path.join(*image_s3_url_components[1:])
    image_f = s3.download_from_s3(bucket, key)
    output_f = overlay_crops_on_image(image_f, processed_anns)
    output_fs.append(output_f)
        


In [ ]:
video_f = os.path.join(OUTPUT_DIR, 'gopro_ann_video.avi')
stitch_frames_into_video(output_fs, video_f)

In [ ]:
OUTPUT_DIR

<h1> Run SORT </h1>

In [ ]:
from sort import Sort

In [ ]:
s = Sort(max_age=100, iou_threshold=0.15)
output_fs = []
count = 0
for idx, row in df.iterrows():
    if count % 10 == 0:
        anns = [ann for ann in row.annotation['annotations'] if ann['category'] == 'HEAD']
        try:
            processed_anns = process_annotated_crops(anns)
        except KeyError:
            s.update()
            trackers_list.append(trackers)
            continue
        X = np.array([list(p['bbox']) + [1.0] for p in processed_anns])
        if len(X) > 0:
            trackers = s.update(X)
        else:
            trackers = s.update()
            
        image_s3_url = row.images[0]
        image_s3_url_components = image_s3_url.replace('s3://', '').split('/')
        bucket, key = image_s3_url_components[0], os.path.join(*image_s3_url_components[1:])
        image_f = s3.download_from_s3(bucket, key)
        output_f = overlay_crops_on_image(image_f, trackers)
        output_fs.append(output_f)
        
    
    count += 1



In [ ]:
count = 0

for idx, row in df.iterrows():
    
    if count % 10 == 0:
        print(count)
    
    trackers = trackers_list[count]

    image_s3_url = row.images[0]
    image_s3_url_components = image_s3_url.replace('s3://', '').split('/')
    bucket, key = image_s3_url_components[0], os.path.join(*image_s3_url_components[1:])
    image_f = s3.download_from_s3(bucket, key)
    output_f = overlay_crops_on_image(image_f, trackers)
    output_fs.append(output_f)
    count += 1
    
    
    


In [ ]:
Image.open(output_f)